In [1]:
import pickle
from parameters.params import Params,Datasets,Models,Tasks
from util.env import set_param
from main import Main
import torch
from models.mine.MSTGAT import MSTGAT
from test_loop import test
from train_loop import train
import pandas as pd
from evaluate import IqrThreshold,MyConfusuion,IqrSensorThreshold,MinMaxThreshold,ZscoreThreshold
import scipy.optimize as opt
import importlib
from parameters.MetricsParameters import MetricsParameters

In [2]:
path = "./snapshot/my_gat_encoder_modal_wadi/25_07_03_23_46_45/0/"

param: Params = pickle.load(file=open(f"{path}param.pickle", "rb"))
param.val_ratio=0
param.stride=1
set_param(param)
model_parameters=pickle.load(file=open( f"{path}model_parameters.pickle","rb"))
adj=torch.load(param.best_validationModel_path().replace("best.pt", "adj.pt"))
for row in adj.int().tolist():
    print("-".join([str(e) for e in row]))

1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
0-0-0-0-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-1-0-0-

C:\Users\moji\AppData\Local\Temp\ipykernel_17100\445764846.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adj=torch.load(param.best_validationModel_path().replace("best

In [ ]:
main = Main(param=param, modelParams=model_parameters, adj=adj)

# main.load_model(path=param.least_trainLossModel_path())

print(len(main.train_dataloader),len(main.val_dataloader),len(main.test_dataloader))
# Train losses
# get all losses from train data loader
train_all_loss,y_train,label_train = test(main.model,main.train_dataloader,None)
print(train_all_loss.shape)
print(label_train.sum())
# Test losses
test_all_losses,ys,lbls = test(main.model, main.test_dataloader,confusion=None)
print(lbls.sum())

In [ ]:
dft=pd.DataFrame({"FN":[],"FP":[],"F1":[],"m":[]})
for m in torch.arange(start=0.8,step=.01,end=1.1):
    mxthr=MinMaxThreshold(multiplier=m)
    mxthr.fit(train_all_loss)
    mpreds=mxthr.transform(test_all_losses)
    mxConf=MyConfusuion(thr=mxthr).cuda()
    mxConf.update(test_all_losses,lbls)
    mtr=MetricsParameters()
    mtr.loadFromConfusion(mxConf.compute())
    dft.loc[len(dft)]=[0,0,mtr.F1,m.numpy()]
dft.plot(x="m")

In [ ]:
def fun(z):
    mxthr=MinMaxThreshold(multiplier=z)
    mxthr.fit(train_all_loss)
    mxConf=MyConfusuion(thr=mxthr).cuda()
    mxConf.update(test_all_losses,lbls)
    mtr=MetricsParameters()
    mtr.loadFromConfusion(mxConf.compute())
    return mtr.FP

result = opt.minimize_scalar(fun, bounds=(0, 3), method='bounded')

In [ ]:
mmthr=ZscoreThreshold(threshold=3.42)
mmthr.fit(train_all_loss)
preds=mmthr.transform(test_all_losses)
mmConf=MyConfusuion(thr=mmthr).cuda()
mmConf.update(test_all_losses,lbls)
mmConf.compute()

In [ ]:
dft=pd.DataFrame({"FN":[],"FP":[],"m":[]})
for m in torch.arange(start=.8,step=.01,end=3.8):
    mxthr=ZscoreThreshold(threshold=m)
    mxthr.fit(train_all_loss)
    mpreds=mxthr.transform(test_all_losses)
    mxConf=MyConfusuion(thr=mxthr).cuda()
    mxConf.update(test_all_losses,lbls)
    mtr=mxConf.compute().cpu().numpy()
    dft.loc[len(dft)]=[mtr[0][1],mtr[1][0],m.numpy()]
dft.plot(x="m")

## AbsMaxThreshold

In [ ]:
from evaluate import AbsMaxThreshold
mmthr=AbsMaxThreshold(multiplier=3.7)
mmthr.fit(train_all_loss)
preds=mmthr.transform(test_all_losses)
mmConf=MyConfusuion(thr=mmthr).cuda()
mmConf.update(test_all_losses,lbls)
mmConf.compute()

In [ ]:

dft=pd.DataFrame({"FN":[],"FP":[]})
for m in torch.arange(start=.5,step=.1,end=5):
    mxthr=AbsMaxThreshold(multiplier=m)
    mxthr.fit(train_all_loss)
    mpreds=mxthr.transform(test_all_losses)
    mxConf=MyConfusuion(thr=mxthr).cuda()
    mxConf.update(test_all_losses,lbls)
    mtr=mxConf.compute().cpu().numpy()
    dft.loc[len(dft)]=[mtr[0][1],mtr[1][0]]
dft.plot()

In [ ]:
from evaluate import BaseThreshold

class testMinMaxThreshold(BaseThreshold):
    """docstring for MinMaxThreshold."""

    def __init__(self, **kwargs):
        super(testMinMaxThreshold, self).__init__(**kwargs)
        self.min = None
        self.max = None

    def fit(self, losses: torch.Tensor):

        self.min = losses.min(dim=0).values * self.multiplier
        self.max = losses.max(dim=0).values * self.multiplier

    def transform(self, loss: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():

            preds = ((loss > self.max) | (loss < self.min)).int().sum(-1)
            preds = torch.where(
                preds > 0,
                1,
                0,
            )
        return preds

In [ ]:
min=train_all_loss.min(dim=0)
max=train_all_loss.max(dim=0)


In [ ]:
# mmthr_class=IqrSensorThreshold
mmthr_class=MinMaxThreshold
multiplier=Main.minimizer(class_type=mmthr_class,train_all_losses=train_all_loss,test_all_losses=test_all_losses,all_labels=lbls)
print(f"multiplier: {multiplier}")
mmthr=mmthr_class(multiplier=multiplier)
mmthr.fit(train_all_loss)
preds=mmthr.transform(test_all_losses)
conf=MyConfusuion(thr=mmthr).to(param.device)
conf.update(test_all_losses,lbls)
conf.compute()

In [ ]:
attack=test_all_losses[3700:3800]
preds = ((attack > mmthr.max) | (attack < mmthr.min))
true_indices = [torch.nonzero(row, as_tuple=False).squeeze(1) for row in preds]

# Output the results
for i, indices in enumerate(true_indices):
    print(f"Batch {i}: True at nodes {indices.tolist()}")

## PLOTTING


all test Datasets

In [ ]:
df=pd.DataFrame({"preds":preds.tolist(),"truth":lbls.tolist()})
df.plot()

In [ ]:
is_one = df['truth'] == 1

# Find start of runs: True where current is 1 but previous is 0 or start of data
start = is_one & (~is_one.shift(1, fill_value=False))

# Find end of runs: True where current is 1 but next is 0 or end of data
end = is_one & (~is_one.shift(-1, fill_value=False))

# Get indices of starts and ends
start_indices = df.index[start]
end_indices = df.index[end]

# Pair them up
runs = list(zip(start_indices, end_indices))

print("Runs of consecutive 1's (start_index, end_index):")
print(runs)

Each attack zone


In [ ]:
import matplotlib.pyplot as plt
from util.data import get_attack_interval,getAttacks


attacks=getAttacks(df,"truth")
# Identify contiguous regions of ones

# Create subplots dynamically
num_regions = len(attacks)
fig, axes = plt.subplots(int((num_regions/2)+.6), 2, figsize=(16, 3*num_regions), sharex=False)

if num_regions == 1:
    axes = [axes]  # Ensure axes is iterable
for i in range(num_regions):
    # first_idx = group.index.min()  # First occurrence of 1
    # last_idx = group.index.max()   # Last occurrence of 1
    row=int(i/2)
    column=i%2
    [first_idx,last_idx]=attacks[i]
    print(f"Attack {i+1} start = {first_idx} end = {last_idx} duration = {last_idx-first_idx}")
    range_before = first_idx - int(2.0 *(last_idx-first_idx))
    range_after = last_idx + int(2.0 *(last_idx-first_idx))
    if range_before<0:
        range_before=0
    if range_after>len(df):
        range_after=last_idx
    # df.iloc[first_idx:last_idx].plot(ax=ax)
    axes[row][column].plot( df.loc[range_before:range_after,'truth'], label="Truth")
    axes[row][column].plot( df.loc[range_before:range_after,'preds'], color='red', label="Preds",linestyle="--")

    # axes[row][column].set_xlim(range_before, range_after)  # X-axis limit dynamically adjusted
    # axes[row][column].set(ybound=(0, max(df['preds']) + 10))  # Corrected method for Y-axis limits

    axes[row][column].set_title(f"Attack {i+1} start = {first_idx} end = {last_idx}")
    axes[row][column].legend()
    axes[row][column].grid(True)

plt.xlabel("Index")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(-.5,.5))
sc.fit(train_all_loss.cpu())

ttest=torch.tensor(sc.transform(test_all_losses.cpu()))*.6
pred=torch.where(ttest>(torch.ones(36)*.5) ,torch.tensor(1),torch.tensor(0))+torch.where(ttest>(torch.ones(36)*.5) ,torch.tensor(1),torch.tensor(0))
pred=pred.sum(-1)
preds=torch.where(pred==0,0,1)

In [ ]:
print("max: ",test_result[0].max(),
    #   val_result[0].max(),
      train_result[0].max())
print("min: ",test_result[0].min(),
    #   val_result[0].min(),
      train_result[0].min())


In [ ]:
from util.data import getAttacks

In [ ]:
minimum=0#val_result[0].min()
maximum=train_result[0].max().item()*1#*24.5

pred=torch.where((test_result[0]>maximum) ,torch.tensor(1),torch.tensor(0)).cpu().numpy()
labels_df=pd.DataFrame({"pred":pred,"truth":test_result[2].cpu().numpy()})
attacks=getAttacks(labels_df,"truth")


In [ ]:

for i,attack in enumerate(attacks):
    left=(attack[0]//100)*100
    right=((attack[1]//100)+1)*100
    labels_df.loc[left:right].plot(title=f"attack {i+1}")

In [ ]:
r=test_result[3].sum(-1)
print(r.max(),r.min(),r.mean())

In [ ]:
from evaluate import createMetrics
k=createMetrics(test_result,maximum)
print(k)
k.keys()

In [ ]:
k2=['TP', 'FP', 'TN', 'FN']
k1=["Accuracy","Precision","Recall","F1"]
d1={key:[] for key in k1}
d2={key:[] for key in k2}
x=[]
for i in range(100):
    _max=maximum*(.5+i/100)
    rr=createMetrics(test_result,_max)
    x.append(_max)
    for key  in d1.keys():
        d1[key].append(float(rr[key]))
    for key  in d2.keys():
        d2[key].append(float(rr[key]))